# Bibliotecas essenciais

Se necessário fazer instalação das bibliotecas não instaladas

In [15]:
pip install yfinance fundamentus tradingcomdados vizro pandas matplotlib

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import matplotlib.pyplot as plt

Bibliotecas para obtenção de dados de MF

In [2]:
import yfinance as yf
import fundamentus

from tradingcomdados import alternative_data as ad
from tradingcomdados import funds_data as fd

import vizro.plotly.express as px
from vizro import Vizro
import vizro.models as vm

2024-07-01 15:01:04,076 [logging.log_init] INFO: LOGLEVEL=INFO
2024-07-01 15:01:06,474 [_charts_utils.<module>] INFO: Overwriting plotly default template with vizro chart template. To revert back to the default template, run `import plotly.io as pio; pio.templates.default = 'plotly'`.


# 1. Importação dos dados

Importando dados do Ibovespa

In [3]:
ibov = ad.index_composition("ibov", mode='list')

Importando os setores da bolsa

In [4]:
setores = ad.get_sectors()
setores.rename(columns={'CÓDIGO':'TICKER','NOME NO PREGÃO':'EMPRESA'}, inplace=True) #inplace=True vai salvar as alterações para tratar depois

Dados fundamentalistas

In [5]:
df = fundamentus.get_resultado()

Cotações

In [6]:
ibov_sa = [i + '.SA' for i in ibov] #Fazer isso pois o yf precisa de .SA para empresas BR

cotacoes_ibov = yf.download(ibov_sa, start = '2023-01-01').Close #.Close é para pegar só os dados de fechamento das cotas

cotacoes_ibov['Date'] = cotacoes_ibov.index

[*********************100%%**********************]  86 of 86 completed
2024-07-01 15:02:05,748 [multi.download] ERROR: 
5 Failed downloads:
2024-07-01 15:02:05,748 [multi.download] ERROR: ['SBSP3.SA']: ReadTimeout(ReadTimeoutError("HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Read timed out. (read timeout=10)"))
2024-07-01 15:02:05,748 [multi.download] ERROR: ['CPFE3.SA', 'KLBN11.SA', 'BBSE3.SA']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
2024-07-01 15:02:05,748 [multi.download] ERROR: ['IGTI11.SA']: YFChartError('%ticker%: Request Failed')


# 2. Processamento dos dados

In [7]:
dados = df[df.index.isin(ibov)]
dados['TICKER'] = dados.index.str[0:4]

C:\Users\thi_t\AppData\Local\Temp\ipykernel_15800\3108437522.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados['TICKER'] = dados.index.str[0:4]


## 2.2 Unindo a tabela de dados fundamentalistas com os setores da bolsa

In [8]:
resultados = pd.merge(dados, setores, how="inner", on='TICKER')

In [9]:
data_melted = pd.melt(cotacoes_ibov,id_vars=['Date'] ,var_name='Ativo',value_name='Price')
data_melted['TICKER'] = data_melted.Ativo.str[0:4]

## 2.3. Jutando dados de cotação com os setores da bolsa

In [10]:
data_quotes = pd.merge(data_melted, setores, how="inner", on='TICKER')
data_quotes['SETOR'] = data_quotes['SEGMENTO'].astype(str)

In [11]:
cotacoes_ibov.drop(['Date'], axis=1, inplace=True)#axis=0 - linhas; axis=1 - colunas

In [12]:
cotacoes_ibov_normalizado = cotacoes_ibov/cotacoes_ibov.iloc[0]
cotacoes_ibov_normalizado['Date'] = cotacoes_ibov_normalizado.index

data_melted_normalizado = pd.melt(cotacoes_ibov_normalizado, id_vars=['Date'], var_name='Ativo', value_name='Price')
data_melted_normalizado['TICKER'] = data_melted_normalizado.Ativo.str[0:4]
data_quotes_normalizado = pd.merge(data_melted_normalizado, setores, how = 'inner', on = 'TICKER')
data_quotes_normalizado['SETOR'] = data_quotes_normalizado['SEGMENTO'].astype(str)

# Dashboard


In [13]:
page1 = vm.Page(
    title = 'Dashboard fundamentalista de ações do Ibovespa',
    components = [
        vm.Graph(id = 'PLvsPVP',figure = px.scatter(resultados, x = 'pl', y = 'pvp', color = 'SETOR ECONÔMICO')),
        vm.Graph(id = 'BAR_ROE',figure = px.bar(resultados, x = 'TICKER', y = 'roe', color = 'SETOR ECONÔMICO')),
        vm.Graph(id = 'BAR_DY',figure = px.bar(resultados, x = 'TICKER', y = 'dy', color = 'SETOR ECONÔMICO'))      
    ],
    controls = [
        vm.Filter(column = 'SEGMENTO'),
        vm.Filter(column = 'SETOR ECONÔMICO'),
        vm.Filter(column = 'pl', selector = vm.RangeSlider()),
        vm.Filter(column = 'dy', selector = vm.RangeSlider()),
        vm.Filter(column = 'pvp', selector = vm.RangeSlider()),
        vm.Filter(column = 'mrgebit', selector = vm.RangeSlider())       
    ]

)

page2 = vm.Page(
    title = 'Cotações de ações do IBOV',
    path = 'IBOV',
    components=[
        vm.Graph(id="cotacoes", figure=px.line(data_quotes, x='Date', y='Price', color="Ativo"))
    ],
    controls = [
        vm.Filter(column = 'SETOR', selector=vm.Checklist())]
    
    
)

page3 = vm.Page(
    title="Cotações de ações do IBOV normalizadas",
    path="IBOV_NORMALIZADO",
    components=[
        vm.Graph(id="cotacoes_normalizadas", figure=px.line(data_quotes_normalizado, x='Date', y='Price', color="Ativo"))
    ],
    controls = [
        vm.Filter(column = 'SETOR', selector=vm.Checklist())]

)

dashboard = vm.Dashboard(pages = [page1, page2, page3])

Vizro().build(dashboard).run()